In [1]:
# import modules
import io, os, sys, types # needed
import glob # needed
from nipype.pipeline.engine import Workflow, Node, MapNode # needed
from nipype.interfaces.utility import Function, IdentityInterface
import nipype.interfaces.io as nio
import nipype.pipeline.engine as pe
from nipype.interfaces.freesurfer import MRIConvert
from nipype.interfaces.freesurfer import ReconAll
from nipype import config

In [2]:
# specify variables
leadsdir = '/cluster/animal/scan_data/leads/'
os.chdir(leadsdir)
dicomdir = "/cluster/animal/scan_data/leads/LEADS/"
unpacklog = "/autofs/cluster/animal/scan_data/leads/recon/unpack.log"
recondir = '/autofs/cluster/animal/scan_data/leads/recon_nip/'
folders = [x for x in os.listdir(dicomdir) if not x.startswith(".")]
subject = 'LDS0370007_20180801'

# wipe clean batch.recon.list
open(recondir+'batch.recon.list', 'w').close()

# subjects with more than one MPRAGE:: these are the better QC ones:
specialcases = {'LDS0370006':'/autofs/cluster/animal/scan_data/leads/LEADS/LDS0370006/Accelerated_Sagittal_MPRAGE/2018-07-26_10_41_42.0/S708999/',
                'LDS0370007':'/autofs/cluster/animal/scan_data/leads/LEADS/LDS0370007/Accelerated_Sagittal_MPRAGE/2018-08-01_10_49_41.0/S711584/',
                'LDS0370013': '/autofs/cluster/animal/scan_data/leads/LEADS/LDS0370013/Accelerated_Sagittal_MPRAGE/2018-08-22_08_55_38.0/S722833/'}

# make a list of one dicom per unique series
alldicomdirs = glob.glob(dicomdir+'/*/Accelerated_Sagittal_MPRAGE/*/*/')
sh_dicomlist = []
for el in alldicomdirs:
    splitstr = el.split('/')
    if "_" not in splitstr[6]:
        file =  os.listdir(el)[0] # pick any dicom
        sh_dicomlist.append(splitstr[6]+'/'+splitstr[7]+'/'+splitstr[8]+'/'+splitstr[9]+'/'+file)

# define workflow
leads_workflow = Workflow(name='leads_workflow') # add  base_dir='/shared_fs/nipype_scratch'

# configure to stop on first crash
cfg = dict(execution={'stop_on_first_crash': True})
config.update_config(cfg)


In [3]:
# sh_dicomlist = [sh_dicomlist[0]]

In [4]:

sh_dicomlist

['LDS0370007/Accelerated_Sagittal_MPRAGE/2018-08-01_10_49_41.0/S711583/LEADS_LDS0370007_MR_Accelerated_Sagittal_MPRAGE__br_raw_20180801153311710_44_S711583_I1029697.dcm',
 'LDS0370007/Accelerated_Sagittal_MPRAGE/2018-08-01_10_49_41.0/S711584/LEADS_LDS0370007_MR_Accelerated_Sagittal_MPRAGE__br_raw_20180801153317254_70_S711584_I1029698.dcm']

In [5]:
# sh_dicomlist = ['LDS0370007/Accelerated_Sagittal_MPRAGE/2018-08-01_10_49_41.0/S711584/LEADS_LDS0370007_MR_Accelerated_Sagittal_MPRAGE__br_raw_20180801153317254_70_S711584_I1029698.dcm',
#  'LDS0370008/Accelerated_Sagittal_MPRAGE/2018-08-15_11_08_29.0/S716587/LEADS_LDS0370008_MR_Accelerated_Sagittal_MPRAGE__br_raw_20180815133149843_198_S716587_I1035748.dcm']

In [6]:
# TEST NODE : PASSSWORDS

def credentials(): # combined with find_dicom
    import getpass
    USER = getpass.getuser()
    #USER= input("Please enter your USERNAME for launchpad acess: ")
    print('Please enter your PASSWORD for launchpad access: ')
    PASS= getpass.getpass()
    return USER, PASS

PASSWORDS = pe.Node(Function(input_names=["user", "pw"],
                         output_names=["USER","PASS"], # actual dicom (redundant to create unpacking node visualization)
                         function=credentials),
                        name='PASSWORDS')


In [7]:
# NODE: CREATEDIR

def createdir(val, USER, PASS):
    import os
    import re
    recondir = '/autofs/cluster/animal/scan_data/leads/recon_nip/' # I think I can make this ./recon_nip/
    subject = val.split('/')[6]
    datestring = val.split('/')[10]
    date = re.search('raw_'+r'+[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]', datestring).group()[4:]
    MPRAGE_path = val.rsplit('/',1)[0]+'/'
    record = True
    while record == True:
        try:
            reconpath = recondir+subject+'_'+date
            imgpath = recondir+subject+'_'+date+'/mri/orig/'
            os.makedirs(imgpath)
            dumplocation = imgpath+'001.mgz'
            dicomlist = os.listdir(MPRAGE_path) #can be any number; choose 0 (no longer need special cases?)
            pickdicom = MPRAGE_path+dicomlist[0]
            record = False
        except(FileExistsError):
            mprage_tmp = MPRAGE_path.split('/')
            mprage_tmp[6] = MPRAGE_path.split('/')[6] + '_' + date
            MPRAGE_path = '/'.join(mprage_tmp) 
            subject = subject+'_'
    return reconpath, MPRAGE_path, pickdicom, dumplocation, recondir, USER, PASS

CREATEDIR = pe.Node(Function(input_names=["val", "USER", "PASS"],
                         output_names=["createdir_out1","createdir_out2", "createdir_out3", "createdir_out4", "createdir_out5", "USER", "PASS"], # actual dicom (redundant to create unpacking node visualization)
                         function=createdir),
                        name='CREATEDIR')


In [8]:
# NODE : UNPACK

def unpack(subjectdir, MPRAGE_path): # combine raw_dump and parse_scaninfo?
    from os import system
    import csv
    import pandas as pd
    cmdstring = 'unpacksdcmdir -src %s -targ %s -scanonly %s/scan.info' % (MPRAGE_path, subjectdir, subjectdir)
    system(cmdstring)
    with open(subjectdir+'/scan.info', 'r') as in_file:
        for line in in_file:
            editline = line.split()
            with open(subjectdir+'/scaninfo.csv', 'w') as result:
                wr = csv.writer(result, dialect='excel')
                wr.writerow(editline)
            result.close()
        in_file.close()
    scan_info = pd.read_csv(subjectdir+'/scaninfo.csv', header=None)
    subname = subjectdir.split('/')[-1]
    return subname, subjectdir, scan_info

UNPACK = pe.Node(Function(input_names=["subjectdir","MPRAGE_path"],
                         output_names=["unpack_out1","unpack_out2", "unpack_out3"], # actual dicom (redundant to create unpacking node visualization)
                         function=unpack),
                        name='UNPACK')


In [9]:
# NODE : CONVERT2MGZ

CONVERT2MGZ = pe.Node(MRIConvert(out_type='mgz'),
                        name='CONVERT2MGZ')

In [10]:
# NODE RENAME_AND_LOG
# note: decided to add this afterward precaution to increase efficiency because there are few errors
# and want to run the unpack and convert2mgz in parallel)

def rename_and_log(subjectdir, scan_info, mgz, reconfolder, subname):
    import re
    import os
    import pandas as pd
    # load in the scaninfo file
    dicomdir = "/cluster/animal/scan_data/leads/LEADS/"
    check = scan_info.iloc[0,2] # first row (only row); second col (validity)
    if check != 'ok':
        with open(reconfolder+'/scanerrors', "a") as efile:
            efile.write(scan_info.iloc[0,7]) # log for errors in dicoms (or any ommitted scans)
            # delete entire recon folder
    else:
        try:                                     # JUST ADDED! rename dicom folder if not already
            oldname = subname.split('_')[0]
            os.rename(dicomdir+oldname, dicomdir+subname)
        except(FileNotFoundError):
            pass
        
        with open(reconfolder+'/batch.recon.list', "a") as bfile:
            bfile.write(subname)
        with open(reconfolder+'/unpack.log', "a") as ufile:
            ufile.write(subname)
        # should I makea scannotes? (will add info after recon)
        Elements = {'scan_notes': [''],'loni_overallpass': [''], 'download_date':[''], 'xnat_upload':[''], 'recon_path':[''],'recon_notes':[''], 'dickerson_overallpass':['']}
        df = pd.DataFrame(Elements, columns= ['scan_notes', 'loni_overallpass', 'download_date','xnat_upload','recon_path','recon_notes','dickerson_overallpass'])
        df.to_csv(subjectdir+'/scannotes.csv')
    return subjectdir, subname

RENAME_AND_LOG = pe.Node(Function(input_names=["subjectdir","scan_info",'mgz', 'reconfolder', 'subname'],
                         output_names=["subjectdir", "subname"],
                         function=rename_and_log),
                        name='RENAME_AND_LOG')

In [11]:
# how can I get username/password to be in function?
# works by using a template and passing in subjectname--just connect this to the nodes

# NODE RECON_JOB

# need to pass p variable into bash from python script.. how ?

def recon_job(subjectname, USER, PASS): # add in username, pass, and subjectname
    from paramiko import SSHClient
    host="launchpad"
    user=USER
    pw=PASS
    client=SSHClient()
    client.load_system_host_keys()
    client.connect(host,username=user,password=pw, look_for_keys=False)
    tmpstr = '(cd /autofs/cluster/animal/scan_data/leads/analyses_nip; setenv p %s ; ./batch.recon.sh)' % subjectname
    stdin, stdout, stderr = client.exec_command(tmpstr)
    #stin = print("stdin: ", stdin.readlines())
    err = "stderr: ", stderr.readlines()
    out = "pwd: ", stdout.readlines()
    if len(err) < 1:
        warning = '0'
    else:
        warning = '1'
    return err, out, warning

RECON_JOB = pe.Node(Function(input_names=["subjectname","USER", "PASS"], 
                        output_names=[ 'err', 'out', 'warning'],
                         function=recon_job),
                        name='RECON_JOB')


In [12]:
# # # NODE : INFOSOURCE
INFOSOURCE = Node(IdentityInterface(fields=['subject_name'], mandatory_inputs=False),
                  name="INFOSOURCE")

INFOSOURCE.iterables = ('subject_name', sh_dicomlist)

# NODE : SELECTFILES
#templates = dict(dicom=sh_dicomlist[0])    ## THIS WORKED!
templates = {
    "dicom": "{subject_name}" 
    }
SELECTFILES = Node(nio.SelectFiles(templates, base_directory=dicomdir),
                   name="SELECTFILES")

# NODE : DATASINK
DATASINK = Node(nio.DataSink(base_directory=leadsdir,
                container='recon_nip'),
                name="DATASINK")

In [13]:
# Connect all nodes (including INFOSOURCE, SELECTFILES, and DATASINK) to workflow

leads_workflow.connect([(INFOSOURCE, SELECTFILES, [('subject_name', 'subject_name')]),
                (SELECTFILES, CREATEDIR, [('dicom', 'val')]),
                (PASSWORDS, CREATEDIR, [('USER', 'USER')]),
                (PASSWORDS, CREATEDIR, [('PASS', 'PASS')]), 
                (CREATEDIR, UNPACK, [('createdir_out1', 'subjectdir')]),
                 (CREATEDIR, UNPACK, [('createdir_out2', 'MPRAGE_path')]),
                 (CREATEDIR, CONVERT2MGZ, [('createdir_out3', 'in_file')]),
                 (CREATEDIR, CONVERT2MGZ, [('createdir_out4', 'out_file')]),   
                (CONVERT2MGZ, RENAME_AND_LOG, [('out_file', 'mgz')]),
                (CREATEDIR, RENAME_AND_LOG, [('createdir_out5', 'reconfolder')]),
                (UNPACK, RENAME_AND_LOG, [('unpack_out1', 'subname')]),
                (UNPACK, RENAME_AND_LOG, [('unpack_out2', 'subjectdir')]),
                (UNPACK, RENAME_AND_LOG, [('unpack_out3', 'scan_info')]),
                (CREATEDIR, RECON_JOB, [('USER','USER')]),     # new
                (CREATEDIR, RECON_JOB, [('PASS','PASS')]),     # new
                (RENAME_AND_LOG, RECON_JOB, [('subname','subjectname')]),         
                (RECON_JOB, DATASINK, [('warning','backup')])  # backup folder?
                 ])

In [14]:
# Execute your workflow in sequential way
# leads_workflow.run(run(plugin='MultiProc', plugin_args={'n_procs' : 2})
leads_workflow.run()

leads_workflow.write_graph(graph2use='flat')

190305-15:04:52,370 nipype.workflow INFO:
	 Workflow leads_workflow settings: ['check', 'execution', 'logging', 'monitoring']
190305-15:04:52,419 nipype.workflow INFO:
	 Running serially.
190305-15:04:52,421 nipype.workflow INFO:
	 [Node] Setting-up "leads_workflow.SELECTFILES" in "/tmp/tmp53_od6tk/leads_workflow/_subject_name_LDS0370007..Accelerated_Sagittal_MPRAGE..2018-08-01_10_49_41.0..S711584..LEADS_LDS0370007_MR_Accelerated_Sagittal_MPRAGE__br_raw_20180801153317254_70_S711584_I1029698.dcm/SELECTFILES".
190305-15:04:52,426 nipype.workflow INFO:
	 [Node] Running "SELECTFILES" ("nipype.interfaces.io.SelectFiles")
190305-15:04:52,437 nipype.workflow INFO:
	 [Node] Finished "leads_workflow.SELECTFILES".
190305-15:04:52,439 nipype.workflow INFO:
	 [Node] Setting-up "leads_workflow.SELECTFILES" in "/tmp/tmpzn1sdyxe/leads_workflow/_subject_name_LDS0370007..Accelerated_Sagittal_MPRAGE..2018-08-01_10_49_41.0..S711583..LEADS_LDS0370007_MR_Accelerated_Sagittal_MPRAGE__br_raw_2018080115331171

190305-15:05:23,615 nipype.interface INFO:
	 stdout 2019-03-05T15:05:23.597624:	InversionTime     900
190305-15:05:23,616 nipype.interface INFO:
	 stdout 2019-03-05T15:05:23.597624:	RepetitionTime    2300
190305-15:05:23,618 nipype.interface INFO:
	 stdout 2019-03-05T15:05:23.597624:	PhEncFOV          240
190305-15:05:23,619 nipype.interface INFO:
	 stdout 2019-03-05T15:05:23.597624:	ReadoutFOV        256
190305-15:05:23,619 nipype.interface INFO:
	 stdout 2019-03-05T15:05:23.597624:Image information
190305-15:05:23,620 nipype.interface INFO:
	 stdout 2019-03-05T15:05:23.597624:	RunNo             1
190305-15:05:23,621 nipype.interface INFO:
	 stdout 2019-03-05T15:05:23.597624:	SeriesNo          2
190305-15:05:23,622 nipype.interface INFO:
	 stdout 2019-03-05T15:05:23.597624:	ImageNo           1
190305-15:05:23,623 nipype.interface INFO:
	 stdout 2019-03-05T15:05:23.597624:	NImageRows        256
190305-15:05:23,624 nipype.interface INFO:
	 stdout 2019-03-05T15:05:23.597624:	NImageCols  

190305-15:06:02,431 nipype.interface INFO:
	 stdout 2019-03-05T15:06:02.431193:INFO: loading series header info.
190305-15:06:02,433 nipype.interface INFO:
	 stderr 2019-03-05T15:06:02.433522:INFO: found 208 files in series
190305-15:06:22,192 nipype.interface INFO:
	 stderr 2019-03-05T15:06:22.191963:
190305-15:06:22,193 nipype.interface INFO:
	 stdout 2019-03-05T15:06:22.193204:INFO: sorting.
190305-15:06:22,194 nipype.interface INFO:
	 stdout 2019-03-05T15:06:22.193204:sdfiSameSlicePos() eps = 0.000001
190305-15:06:22,195 nipype.interface INFO:
	 stdout 2019-03-05T15:06:22.193204:INFO: (240 256 208), nframes = 1, ismosaic=0
190305-15:06:22,196 nipype.interface INFO:
	 stderr 2019-03-05T15:06:22.196533:RunNo = 5
190305-15:06:22,200 nipype.interface INFO:
	 stdout 2019-03-05T15:06:22.200251:sdfi->UseSliceScaleFactor 0
190305-15:06:22,201 nipype.interface INFO:
	 stdout 2019-03-05T15:06:22.200251:datatype = 4, short=4, float=3
190305-15:06:22,202 nipype.interface INFO:
	 stdout 2019-03

190305-15:06:47,230 nipype.workflow INFO:
	 [Node] Setting-up "leads_workflow.RECON_JOB" in "/tmp/tmpvfuiehwi/leads_workflow/_subject_name_LDS0370007..Accelerated_Sagittal_MPRAGE..2018-08-01_10_49_41.0..S711583..LEADS_LDS0370007_MR_Accelerated_Sagittal_MPRAGE__br_raw_20180801153311710_44_S711583_I1029697.dcm/RECON_JOB".
190305-15:06:47,236 nipype.workflow INFO:
	 [Node] Running "RECON_JOB" ("nipype.interfaces.utility.wrappers.Function")
190305-15:06:58,728 nipype.workflow INFO:
	 [Node] Finished "leads_workflow.RECON_JOB".
190305-15:06:58,730 nipype.workflow INFO:
	 [Node] Setting-up "leads_workflow.DATASINK" in "/tmp/tmp26qcw02m/leads_workflow/_subject_name_LDS0370007..Accelerated_Sagittal_MPRAGE..2018-08-01_10_49_41.0..S711583..LEADS_LDS0370007_MR_Accelerated_Sagittal_MPRAGE__br_raw_20180801153311710_44_S711583_I1029697.dcm/DATASINK".
190305-15:06:58,743 nipype.workflow INFO:
	 [Node] Running "DATASINK" ("nipype.interfaces.io.DataSink")
190305-15:06:58,754 nipype.workflow INFO:
	 [No

'/autofs/cluster/animal/scan_data/leads/graph.png'

In [15]:
leads_workflow.write_graph(graph2use='flat')

190305-15:06:59,842 nipype.workflow INFO:
	 Generated workflow graph: /autofs/cluster/animal/scan_data/leads/graph.png (graph2use=flat, simple_form=True).


'/autofs/cluster/animal/scan_data/leads/graph.png'